In [1]:
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from tqdm import tqdm
from tqdm import trange
import jsonlines
from os import listdir
import itertools
import re
from collections import Counter
import json
import pymorphy2 as pm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix
from scipy.sparse import save_npz
from scipy.sparse import load_npz

In [2]:
ok_set = set()
with open("sources/accepted_categories.txt", mode="r", encoding="utf-8") as inp:
    for line in inp:
        line = line[:-1]
        ok_set.add(line)

ok = sorted(ok_set)

categories_dict = {}
with open("sources/article_cat.json", mode="r") as input:
    categories_dict = json.loads(input.read())

In [3]:
ids = sorted([item['id'] for item in jsonlines.open('sources/normalized_texts.jl', 'r')])
X = np.load("sources/svd500.npy")

In [4]:
kf = KFold(n_splits=3, shuffle=True, random_state=27)


train_index, test_index = 0, 0
for item in kf.split(X):
    train_index, test_index = item[0], item[1]
    break
    
print("TRAIN:", train_index, "TEST:", test_index)
print(len(train_index), len(test_index))
    
X_train = X#[train_index]
ids_train = ids#[ids[i] for i in train_index]

class_centroids = []
skipped_categories = []

for i, category in enumerate(tqdm(ok)):
    clf_centroids = NearestCentroid()
    y_train = np.array([1 if category in categories_dict[id] else 0 for id in ids_train])
    if sum(y_train) > 0:
        clf_centroids.fit(X_train, y_train)
        class_centroids.append(clf_centroids.centroids_[1])
    else:
        skipped_categories.append(ok[i])
        print(category)   

TRAIN: [    0     1     2 ... 96790 96792 96793] TEST: [    3     4     7 ... 96785 96789 96791]
64529 32265


100%|██████████████████████████████████████████████████████████████████████████| 14697/14697 [1:43:55<00:00,  2.36it/s]


In [5]:
class_centroids[0].shape

(500,)

In [6]:
class_centroids = np.array(class_centroids)

In [7]:
np.save("sources/centroids_svd500", class_centroids)